In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import time
import mysql.connector
from tqdm.notebook import tqdm

In [2]:
def connect_db():
    db = mysql.connector.connect(
        host="101.101.210.141",
        user="study",
        passwd="study!@#$%",
        database="seoul_welfare"
        # database="aihub_seoul_healthcare"
)
    return db

def mysql_query(database, query:str):
    database.reconnect()
    cur = database.cursor() 
    cur.execute(query)
    result = cur.fetchall()

    return result

db = connect_db()

In [3]:
train_data_path = "../dpo/data/0514university_dpo_row.jsonl"
dataset = load_dataset("json", data_files = train_data_path, split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'accept', 'reject', 'context'],
    num_rows: 398
})

### SQL Dataset

In [3]:
train_data_path = "../data/sql_context/sft_train.jsonl"
dataset = load_dataset("json", data_files = train_data_path, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
eval_data_path = '../data/sql_context/dpo_train.jsonl'
evalset = load_dataset("json", data_files = eval_data_path, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
print(f"train: {dataset}\nval: {evalset}")

train: Dataset({
    features: ['input', 'context', 'output'],
    num_rows: 7742
})
val: Dataset({
    features: ['input', 'context', 'output'],
    num_rows: 5068
})


### Welfare

In [3]:
train_data_path = "../data/welfare/sft_train.jsonl"
dataset = load_dataset("json", data_files = train_data_path, split="train")
eval_data_path = '../data/welfare/test.jsonl'
evalset = load_dataset("json", data_files = eval_data_path, split='train')
print(f"train: {dataset}\nval: {evalset}")

train: Dataset({
    features: ['input', 'context', 'output'],
    num_rows: 1216
})
val: Dataset({
    features: ['input', 'context', 'output'],
    num_rows: 136
})


### Fine Tuning

In [4]:
max_seq_length = 2048
dtype = torch.float16
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.642 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model.config.to_dict()["torch_dtype"]

'float16'

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, 
    bias = "none", 
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
prompt_1 = """### Task
Generate a SQL query to answer the following question:
`{}`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"{}"
 
### SQL
Given the database schema, here is the SQL query that answers `{}`:
[SQL]{}"""

EOS_TOKEN = tokenizer.eos_token

def format_prompt(sample):
    input       = sample["input"]
    context     = sample["context"]
    output      = sample["output"] + EOS_TOKEN

    sample["prompt"]   = prompt_1.format(input, context, input, output)
    return sample
pass

dataset = dataset.map(format_prompt,)
# dataset = dataset.remove_columns(['input', 'accept', 'reject', 'context'])

In [8]:
dataset

Dataset({
    features: ['input', 'context', 'output', 'prompt'],
    num_rows: 1216
})

In [9]:
print(dataset['prompt'][1])

### Task
Generate a SQL query to answer the following question:
`이름이 드림홈스쿨인 시설의 종류명과 상세 종류명을 알려줘`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"CREATE TABLE fclt_open_info_g_c (  GC_WELFARE_FCLT_NM varchar(26) DEFAULT NULL,  GC_WELFARE_FCLT_CD varchar(11) DEFAULT NULL,  GC_WELFARE_FCLT_KIND_NM varchar(28) DEFAULT NULL,  FCLT_KIND_DTL_NM varchar(16) DEFAULT NULL,  GC_WELFARE_JRSD_SGG_CD bigint DEFAULT NULL,  FCLT_ADDR varchar(59) DEFAULT NULL) ;"
 
### SQL
Given the database schema, here is the SQL query that answers `이름이 드림홈스쿨인 시설의 종류명과 상세 종류명을 알려줘`:
[SQL]SELECT GC_WELFARE_FCLT_KIND_NM, FCLT_KIND_DTL_NM FROM FCLT_OPEN_INFO_G_C WHERE GC_WELFARE_FCLT_NM = '드림홈스쿨'<|end_of_text|>


In [10]:
from datetime import datetime
import wandb, os
wandb.login()

model_name = "meta-llama/Meta-Llama-3-8B"  
wandb_project = "llama-3-university-sft"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
    
project = "Qlora-4bit"
run_name = model_name + "-" + project

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: crysis. Use `wandb login --relogin` to force relogin


In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs = 3,
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_grad_norm = 0.3,
        max_steps = -1,
        learning_rate = 2e-4,
        # fp16 = not torch.cuda.is_bf16_supported(),
        # bf16 = torch.cuda.is_bf16_supported(),
        fp16 = True,
        bf16 = False,
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
        logging_strategy = 'steps',
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
)

In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.642 GB.
6.066 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,216 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 114
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,1.729700
2,1.702700
3,1.666600
4,1.439200
5,1.212600
6,0.957900
7,0.736300
8,0.663600
9,0.625200
10,0.676200


In [25]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
i = 5
inputs = tokenizer(
[
    alpaca_prompt.format(
        f"{evalset[i]['input']}", # input,
        f"{evalset[i]['context']}",
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [29]:
excute_result = []
query_cnt_result = []
include_query_result = []
cnt_query_result = []
prompt_list = []
inference_query = []
fail_check = []
FastLanguageModel.for_inference(model) 

In [ ]:
for i in tqdm(range(len(evalset))):
    prompt = alpaca_prompt.format(
            f"{evalset[i]['input']}",
            f"{evalset[i]['context']}",
            "",
        )
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{evalset[i]['input']}",
            f"{evalset[i]['context']}",
            "", 
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
    result = tokenizer.batch_decode(outputs)
    result = result[0].split("### Response:")[-1].split("### Explanation:")[0].split("<|end_of_text|>")[0].replace("\n", "")
    torch.cuda.empty_cache()
    inference_query.append(result)
    prompt_list.append(prompt)
    
    try:
        infer_query_result = mysql_query(db, result)
        real_query_result = mysql_query(db, evalset['output'][i])
        comp = any(item in real_query_result for item in infer_query_result)
        excute_result.append(1)
        cnt_query_result.append(len(infer_query_result))
        if len(infer_query_result) == len(real_query_result):
            query_cnt_result.append(1)
        else:
            query_cnt_result.append(0)
             
        if comp == True:
            include_query_result.append(1)
        else:
            include_query_result.append(0)
    except Exception as e:
        excute_result.append(0)
        cnt_query_result.append(0)
        print(i, e)
        fail_check.append(i)

    time.sleep(0.5)

In [36]:
print(f"쿼리 실행 확률 : {round(excute_result.count(1) / len(evalset), 2)}\n\
쿼리 결과 수 같을 확률 : {round(query_cnt_result.count(1) / len(evalset), 2)}\n\
쿼리 결과를 포함될 확률 : {round(include_query_result.count(1) / len(evalset), 2)}")

쿼리 실행 확률 : 0.98
쿼리 결과 수 같을 확률 : 0.71
쿼리 결과를 포함될 확률 : 0.7


In [45]:
import pandas as pd
prompt_ = pd.Series(prompt_list)
rejected = pd.Series(inference_query)
chosen = pd.Series(evalset['output'])

df = pd.concat([prompt_, chosen, rejected], axis=1)
df.rename(columns={ 0: 'prompt', 1: 'chosen', 2: 'rejected'}, inplace = True)
df.to_csv("../dpo/data/dpo_data.csv")


In [22]:
import pandas as pd
train_excel = pd.DataFrame(dataset)
train_excel.to_excel('./data/welfare_train.xlsx')

val_excel = pd.DataFrame(evalset)
val_excel.to_excel('./data/welfare_val.xlsx')

### save

In [14]:
# model.save_pretrained("llama3-8B-welfare-rollback") # Local saving
model.push_to_hub("Crysiss/llama-3-8B-welfare-sft-f16") # Online saving

README.md:   0%|          | 0.00/574 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Crysiss/llama-3-8B-welfare-sft-f16
